In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np


In [36]:
class_labels = ['Red', 'Trout', 'Flounder','largemouth_bass','smallmouth_bass']
# Load and Preprocess Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'data/train',
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    classes=class_labels  # List the class names explicitly
)

Found 685 images belonging to 5 classes.


In [37]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [38]:
model.fit(train_generator, epochs=50, batch_size=16)


Epoch 1/50


/Users/albertoalvarado/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 609ms/step - accuracy: 0.2443 - loss: 2.9968
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 604ms/step - accuracy: 0.3275 - loss: 1.5088
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 16s 572ms/step - accuracy: 0.3431 - loss: 1.4991
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 16s 565ms/step - accuracy: 0.4246 - loss: 1.3938
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 592ms/step - accuracy: 0.4236 - loss: 1.3914
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 622ms/step - accuracy: 0.4177 - loss: 1.3895
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 602ms/step - accuracy: 0.4020 - loss: 1.3696
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 581ms/step - accuracy: 0.4899 - loss: 1.3308
Epoch 9/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 597ms/step - accuracy: 0.4545 - loss: 1.3222
Epoch 10/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 600ms/step - accuracy: 0.4390 - loss: 1.3074
Epoch 11/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 628ms/step - accuracy: 0.4278 - loss: 1.3596
Epoch 12/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 598ms/ste

In [39]:
model.save('fish_identifier_model.h5')


In [44]:
# Prediction Function
def predict_fish(image_path, model):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    class_labels = ['Red', 'Trout', 'Flounder','largemouth_bass','smallmouth_bass']

    predicted_class = class_labels[predicted_class_index]
    confidence = prediction[0][predicted_class_index]

    
    return predicted_class, confidence

# Example usage:
image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/UTSA.Lessons/Fish-Species-Identification-CNN/Project4/FishExamples/testtrout.png'
#image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/UTSA.Lessons/Fish-Species-Identification-CNN/Project4/FishExamples/flounder_test.jpeg'
#image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/UTSA.Lessons/Fish-Species-Identification-CNN/Project4/FishExamples/red_test.jpeg'
#image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/UTSA.Lessons/Fish-Species-Identification-CNN/Project4/FishExamples/largemouth_test.png'
#image_path = '/Users/albertoalvarado/Documents/utsa/UTSA.Lessons/UTSA.Lessons/Fish-Species-Identification-CNN/Project4/FishExamples/smallmouth_test.png'

predicted_fish_class,confidence = predict_fish(image_path, model)
print("Predicted fish class:", predicted_fish_class)
print("Confidence:", confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted fish class: Red
Confidence: 0.4814885


In [9]:
import requests
from io import BytesIO

def predict_fish_from_url(image_url, model):
    response = requests.get(image_url)
    img = image.load_img(BytesIO(response.content), target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    
    # Assuming you have access to the class labels
    class_labels = ['Red', 'Trout', 'Flounder','largemouth_bass','smallmouth_bass']

    
    predicted_class = class_labels[predicted_class_index]
    
    confidence = prediction[0][predicted_class_index]

    
    return predicted_class, confidence

# Example usage:
image_url = 'https://www.usm.edu/ocean-science-engineering/images/red-drum.jpg'
predicted_fish_class,confidence = predict_fish_from_url(image_url, model)
print("Predicted fish class:", predicted_fish_class)
print("Confidence:", confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted fish class: Red
